# Proyecto Angular

Otra forma de entender un framework es poniendo en práctica los conceptos. Haremos un proycto sencillo con la mayor parte de lo aprendido.

En el curso hemos hecho una aplicación para compartir recetas. Documentaremos algunas partes para este artículo, evitando las tareas repetitivas. 


## Configuración inicial

Como en todo proyecto, hay una tarea prévia de configuración. No es necesario explicar todos los pasos o el porqué, ya que está en apartados anteiores, pero vamos a enumerarlos. Algunos pasos, lógicamente, no son necesarios en todos los proyectos:

* Creación de un repositorio de Git y las ramas, issues, etc. 
* Instalación de Angular CLI y creación del proyecto Angular:
```bash
sudo npm install -g @angular/cli [--force]
ng new recipes
cd recipes 
ng serve -o
```
* Creación de todos los componentes, interfaces y servicios que se han decidido durante la etapa del análisis:

```bash
ng g component components/header
ng g component components/footer
ng g component components/home
ng g component components/login
ng g component recipes/recipe-card
ng g component recipes/recipe-detail
ng g component recipes/recipe-table-row
ng g component recipes/recipe-table
ng g component recipes/recipe-list
...
ng g service recipes/supabase
ng g interface recipes/i-recipe
...
```
> Es buena idea crear todos los componentes que sepamos al principio para tener más claro la estructura y para no tener que reiniciar cada vez. 
* Creación de la estructura básica de la web en `app.component.html`:

```html
<app-header></app-header>
<router-outlet></router-outlet>
<app-footer></app-footer>
```
* Creación de las rutas y testeo manual en `app.routes.ts`:
```javascript
export const routes: Routes = [
    {path: 'home', component: HomeComponent},
    {path: 'main', component: RecipeListComponent},
    {path: 'table', component: RecipeTableComponent},
    {path: 'recipes/:id', component: RecipeDetailComponent},
    {path: '**', pathMatch: 'full', redirectTo: 'home'}
];
```
En `app.config.ts`:

```javascript
provideRouter(routes,  withHashLocation()),
```

* Instalación de `Bootstrap`:
```bash
npm install bootstrap
npm install --save-dev @types/bootstrap
```
en `angular.json`:
```javascript
"styles": [
  "node_modules/bootstrap/dist/css/bootstrap.min.css",
  "src/styles.scss"
],
"scripts": [
  "node_modules/bootstrap/dist/js/bootstrap.bundle.min.js"
]
```
* Creación de un `Navbar` de Bootstrap para el menú dentro del componente `header`. Cada ruta se indicará así:

```html
<a class="nav-link" aria-current="page" [routerLink]="['home']" [routerLinkActive]="['active']">Home</a>
```
* Creación del HTML estático de secciones como el `footer` o `home`.

Así tendremos la estructura mínima con rutas de la `SPA` y ya podemos empezar a trabajar en la funcionalidad y la vista de la web. 

> Los pasos anteriores no detallan la importación de los módulos en los archivos y otros pasos. Todo esto está explicado en apartados anteriores de este libro. 

## Componentes con datos de ejemplo

Empezaremos dando contenido a los componentes. Un buen ejemplo puede ser la lista de recetas. 

### Interfaz IRecipe

Aquí tenemos un fragmento de la interfaz, que se corresponde con las columnas de la tabla en la base de datos:

```javascript
export interface IRecipe {
    idMeal:                      null | string;
    strMeal:                     null | string;
    strDrinkAlternate:           null | string;
    strCategory:                 null | string;
    strArea:                     null | string;
    strInstructions:             null | string;
    strMealThumb:                null | string;
    strTags:                     null | string;
    strYoutube:                  null | string;
    strIngredient1:              null | string;
    strIngredient2:              null | string;
    ...
```

### Lista de recetas

El componente `RecipesListComponent` debe tener algo que mostrar. Al principo es posible hacer un `mock`de los datos creando un array de recetas que respeta la interfaz creada anteriormente:

```javascript
import { IRecipe } from "../i-recipe"

export const recipes: IRecipe[] = [
    {
        "idMeal": "52878",
        "strMeal": "Beef and Oyster pie",
        "strDrinkAlternate": null,
        "strCategory": "Beef",
```

Este mock será descartado cuando tengamos los datos de la base de datos, pero puede servir mientras tanto para crear los componentes.

El primer componente será `RecipesListComponent`. Aquí un extracto del código relevante:

```javascript
import {recipes} from "./recipes_exemples"

@Component({
  selector: 'app-recipes-list',
  imports: [CommonModule, RecipeCardComponent],
  templateUrl: './recipes-list.component.html',
  styleUrl: './recipes-list.component.css'
})
export class RecipesListComponent implements OnInit, OnDestroy {

  public recipes: IRecipe[] = recipes;
```

En el HTML podemos hacer un `@for` para ir creando los `<app-recipe-card>` para cada receta:

```html
@for (recipe of recipes; track $index) {
<div class="col">
    <app-recipe-card [recipe]="recipe"></app-recipe-card>
</div>
}
```

#### @Input con las tarjetas

Como se ve, en cada `<app-recipe-card>` se añade `[recipe]="recipe"` para pasar ese objeto por `@Input` al componente hijo. Este lo recibe así:

```javascript
 @Input({ required: true,  }) recipe!: IRecipe;
 ```

Le hemos llamado tarjetas porque las podemos hacer con un `card` de Bootstrap:

```html
 <div class="card" style="width: 18rem;">
        <img src={{recipe.strMealThumb}} class="card-img-top" alt={{recipe.strMeal}}>
        <div class="card-body">
          <h5 class="card-title">{{recipe ? recipe.strMeal : "Sin titulo"}}</h5>
          <p>{{recipe.strInstructions}}</p>
          <a [routerLink]="['/recipes',recipe.idMeal]"  class="btn btn-primary">Detail</a>
        </div>
      </div>
```

#### @Output de las tarjetas a la vista

Vamos a poner algún tipo de retroacción a las tarjetas para demostrar el funcionamiento de `@Output` y lo dejaremos listo para cuando podamos guardarla de forma persistente en la base de datos. 

### La tabla de recetas

En el caso de la tabla hemos hecho el componente `RecipeTableComponent` que contendrá los componentes `RecipeTableRowComponent`. El funcionamiento es igual que con las card, pero el problema es que `<table>` debe tener siempre `<tr>` inmediatamente dentro. Por eso no podemos crear una etiqueta para el componente `RecipeTableRowComponent`. Para solucionarlo usaremos un selector por atributo: 

```javascript
@Component({
  selector: '[app-recipe-table-row]',
  imports: [],
  templateUrl: './recipe-table-row.component.html',
  styleUrl: './recipe-table-row.component.css'
})
```

Y lo añadiremos como atributo de los tr:

```html
@for (recipe of recipes; track $index) {
<tr app-recipe-table-row [recipe]="recipe"></tr>
}
```

## Servicios

Necesitamos servicios para gestionar la comunicación con Supabase de los distintos componentes.

### Configurar Supabase

En este caso vamos a usar el `SDK` de Supabase por simplificar. Este SDK es genérico y no totalmente enfocado a la metodología `Angular` de usar el `HTTPClient` y `Observables`. Por eso vamos a adaptar las peticiones a Observables. 

Como se ve en el apartado de Supabase y Angular, hay que crear el `environment`:

```bash
ng generate environments 
```

Y añadir los datos de conexión con Supabase:

```javascript
export const environment = {
  production: false,
  supabaseUrl: 'YOUR_SUPABASE_URL',
  supabaseKey: 'YOUR_SUPABASE_KEY',
};
```

Luego en el servicio haremos las peticiones:

```javascript
@Injectable({
  providedIn: 'root'
})
export class SupabaseService {

  private supabase: SupabaseClient;

  constructor(private http: HttpClient) { 
    this.supabase = createClient(environment.supabaseUrl, environment.supabaseKey);
  }

// Función genérica para Supabase
  async getData(table: string): Promise<any[]> {
    const { data, error } = await this.supabase.from(table).select('*');
    if (error) {
      console.error('Error fetching data:', error);
      throw error;
    }
    return data;
  }

// Adaptador de las promesas a Observables
  getDataObservable(table: string): Observable<any[]> {
    return from(this.getData(table));
  }

// Función que hace la petición a una tabla en concreto
  getMeals(): Observable<IRecipe[]>{
    return this.getDataObservable('meals');
  }
```

### Convertir Supabase SDK en Observables

Con las promesas resultantes haremos un Observable:

```javascript
// Adaptador de las promesas a Observables
  getDataObservable(table: string): Observable<any[]> {
    return from(this.getData(table));
  }

// Función que hace la petición a una tabla en concreto
  getMeals(): Observable<IRecipe[]>{
    return this.getDataObservable('meals');
  }
```

### Mostrar los datos en la lista de recetas

Puesto que `getMeals` retorna una Observable de listas de recetas, en el componente `RecipesListComponent` nos suscribimos y quitamos las recetas de ejemplo:

```javascript
export class RecipesListComponent implements OnInit, OnDestroy {

  constructor(private supabaseService: SupabaseService){}

  public recipes: IRecipe[] = [];

  ngOnInit(): void {

    this.supabaseService.getMeals().subscribe({
      next: meals => {
       this.recipes = meals;
      },
      error: err => console.log(err),
      complete: ()=> console.log('Received')
    })
  }
```



## Rutas con parámetros

Antes ya hemos configurado el router para tener rutas con parámetros a una receta en particular:

```javascript
    {path: 'recipes/:id', component: RecipeDetailComponent},
```

### Crear e invocar rutas con parámetros

Ahora hay que saber llamar a esas recetas. En el componente `RecipeCardComponent` hemos puesto un botón con este enlace:

```html
<a [routerLink]="['/recipes',recipe.idMeal]" class="btn btn-primary">Detail</a>
```

### Aceptar los parámetros en las rutas

La manera de aceptar los parámetros en las rutas puede ser mediante `withComponentInputBinding`. En `app.config.ts`:

```javascript
 provideRouter(routes,  withHashLocation(), withComponentInputBinding()),
```

Y luego en el componente `RecipeDetailComponent` las aceptamos como `@Input`:

```javascript
@Input('id') recipeID?: string;
```

### Obtener y mostrar la receta

Una vez hecho esto, hay que pedir la receta, así que necesitamos:

#### Función en SupabaseService para obtener una receta

En este caso lo que hemos hecho es modificar las funciones para hacerlas más genéricas y añadir la posibilidad de pedir más de un dato:

```javascript
async getData(table: string, search?: Object, ids?: string[], idField?: string): Promise<any[]> {
    let query = this.supabase.from(table).select('*');
    if (search) {
      query = query?.match(search);
    }
    if (ids) {
      console.log(idField);

      query = query?.in(idField ? idField : 'id', ids);
    }
    const { data, error } = await query
    if (error) {
      console.error('Error fetching data:', error);
      throw error;
    }
    return data;
  }

getDataObservable<T>(table: string, search?: Object, ids?: string[], idField?: string): Observable<T[]> {
    return from(this.getData(table, search, ids, idField));
  }

getIngredients(ids: (string | null)[]): Observable<Ingredient>{
    return this.getDataObservable<Ingredient>('ingredients', undefined, ids.filter(id => id !== null) as string[], 'idIngredient')
    .pipe(
      mergeMap(ingredients =>
        from(ingredients).pipe(
          mergeMap(async ingredient => {
            const { data, error } = await this.supabase
              .storage
              .from('recipes')
              .download(`${ingredient.strStorageimg}?rand=${Math.random()}`);
            if (data) {
              ingredient.blobimg = URL.createObjectURL(data);
            }
            return ingredient;
          })
        )
      )
    );
  }
```

El operador mergeMap se utiliza en este caso para procesar de manera asíncrona cada ingrediente y emitir cada uno tan pronto como se haya completado el procesamiento.

El primer mergeMap Toma el array de ingredientes emitido por getDataObservable y lo convierte en un flujo de datos que permitirá emitir ingrediente por ingrediente. El from descompone el array para ir trabajando con todos. El mergeMap de dentro convierte las promesas de la descarga de cada imagen en un observable cada iteración del from. Al final se retornan los ingredientes con la imagen dentro de un flujo en el que cada ingrediente va saliendo conforme se completa.

#### Obtener el Observable en ngOnInit

```javascript
ngOnInit(): void {
    this.supabaseService.getMeals(this.recipeID).subscribe({
      next: meals => {
       this.recipe = meals[0];
      this.supabaseService.getIngredients(this.recipe?.idIngredients).subscribe({
        next: ingredients => {
          this.ingredients.push(ingredients);
        }
      });
      },
      error: err => console.log(err),
      complete: ()=> console.log('Received')
    })
  }
```

Así se van incorporando los ingredientes cuando van llegando.

#### Mostrar la receta en la plantilla

```html
<div class="container">
  <div class="row">
    <div class="col-md-6">
      <h2>{{recipe?.strMeal}}</h2>
      <h3>Instructions</h3>
      <p>{{recipe?.strInstructions}}</p>
      <h3>Ingredients</h3>
      <div class="row row-cols-1 row-cols-md-3 g-4">
        @for (ingredient of ingredients; track $index) { @if (ingredient) {
        <div class="col">
          <app-ingredient [ingredient]="ingredient"></app-ingredient>
        </div>
        } }
      </div>
    </div>
    <div class="col-md-6">
      <img
        src="{{recipe?.strMealThumb}}"
        alt="{{recipe?.strMeal}}"
        class="img-fluid"
      />
    </div>
  </div>
</div>
```


## Autenticación

Puesto que el SDK tiene su propia manera de autenticar y mantener la sesión, no vamos a necesitar guardar el token en `LocalStorage` ni otros métodos manuales. No obstante, hay que crear un `Guard` para evitar las rutas no permitidas. Tampoco hace falta el `Interceptor`, ya que todas las peticiones se hacen con el SDK y ya tienen un mecanismo equivalente.

### Funciones de Register, Login, Logout
### Componentes para el registro y acceso
### Estado de la sesión
### Ocultar menús
### Guards


## Formularios

En el apartado de autenticación hemos usado un formulario de plantilla y en este vamos a usar uno `Reactivo`. 

### Validadores
### Cargar datos en el formulario
### Formulario dinámico
### Funciones Supabase para crear y editar recetas


## Aplicación en tiempo real

### Supabase Realtime
### Websockets con SDK
### Websockets a Observables
### Interfaz

## Extra: Estilos personalizados de Bootstrap

Supongamos que estamos usando Bootstrap y queremos unos colores personalizados para toda la aplicación. Puesto que funciona con `Sass`, podemos modificar el valor de variables y maps. 

Quitaremos la referencia a Bootstrap de `angular.json` y la añadiremos al `styles.scss`: 

```css
// Definir colores personalizados

@use "sass:map";
$primary: #25408f;
$secondary: #8f5325;
$success: #3e8d63;
$info: #13101c;
$warning: #945707;
$danger: #d62518;
$light: #061625;
$dark: #343a40;


$theme-colors: (
  primary: $primary,
  secondary: $secondary,
  success: $success,
  info: $info,
  warning: $warning,
  danger: $danger,
  light: $light,
  dark: $dark,
);



// definir colores customizados
$custom-colors: (
  "brand-blue": #2EC4B6,
  "brand-orange": #FF9F1C,
  "brand-orange-light": #FFBF69,
  "brand-blue-light": #CBF3F0

);

// Combina las paletas
$theme-colors: map.merge($theme-colors, $custom-colors);


// importar finalmente Bootstrap para todo lo demás.
@import "../node_modules/bootstrap/scss/bootstrap" 
```

Luego podemos usar esos colores personalizados en otras partes, como en el `navbar`: 

```html
<nav class="navbar navbar-expand-lg bg-brand-blue">
```

Los colores customizados los hemos obtenido de esta paleta: https://coolors.co/palette/ff9f1c-ffbf69-ffffff-cbf3f0-2ec4b6